In [11]:
import numpy as np
import pandas as pd
import scipy.stats
import itertools
import random

In [12]:
data = pd.read_csv('firmness.txt', sep = ",")
#data = data.rename(columns = {"softness(mon)": "mon", "softness(wed)": "wed"})
data

,number,treatment,softness(wed),softness(mon)
0,1,0,3.0,1.0
1,2,0,4.0,2.0
2,3,3,4.0,1.0
3,4,3,4.0,2.0
4,5,1,4.0,2.0
5,6,2,4.0,1.0
6,7,2,4.0,3.0
7,8,2,5.0,2.0
8,9,3,3.0,2.0
9,10,0,4.0,2.0


In [13]:
data.insert(4, column = "difference", value = data.iloc[:,2] - data.iloc[:,3])

In [14]:
control = data[data.iloc[:,1] == 0]
group1 = data[data.iloc[:,1] == 1]
group2 = data[data.iloc[:,1] == 2]
group3 = data[data.iloc[:,1] == 3]

In [15]:
def permutation_test(df1, df2):
    ls = [1 for i in range(df1.shape[0])]
    for i in range(df2.shape[0]):
        ls.append(0)
    permutations = list(itertools.permutations(ls))
    T = df2["difference"].mean() - df2["difference"].mean()
    hist = []
    combined_data = pd.concat([df1,df2])
    for perm in permutations:
        group0 = combined_data[np.asarray(perm)==0]
        group1 = combined_data[np.asarray(perm) == 1]
        hist.append(group1["difference"].mean() - group0["difference"].mean())
    return hist, np.mean(hist >= T)
    
    

In [16]:
def F_test(df1,df2,df3,df4):
    def BGV(df1,df2,df3,df4):
        Ybar = pd.concat([df1,df2,df3,df4])["difference"].mean()
        BGV = (df1["difference"].mean() - Ybar)**2 + (df2["difference"].mean() - Ybar)**2 +(df3["difference"].mean() - Ybar)**2+ (df4["difference"].mean() - Ybar)**2
        return BGV
    def WGV(df1,df2,df3,df4):
        WGV = df1["difference"].var() + df2["difference"].var()+ df3["difference"].var()+df4["difference"].var()
        return WGV / 4   
    combined_data = pd.concat([df1,df2,df3,df4])
    F = BGV(df1,df2,df3,df4)/WGV(df1,df2,df3,df4)
    #print(F)
    ls = [0,0,0,1,1,1,2,2,2,3,3,3]
    permutations = random.choices(list(itertools.permutations(ls)), k = 1000)
    #combined_data = pd.concat([df1,df2,df3,df4])
    hist = []
    for perm in permutations:
        group0 = combined_data[np.asarray(perm)==0]
        group1 = combined_data[np.asarray(perm) == 1]
        group2 = combined_data[np.asarray(perm) == 2]
        group3 = combined_data[np.asarray(perm)==3]
        if WGV(group0,group1,group2,group3) != 0:
            hist.append( BGV(group0,group1,group2,group3) / WGV(group0,group1,group2,group3))
    return np.mean(hist >= F), hist
        

In [17]:
#test = F_test(control, group1, group2, group3)